<a href="https://colab.research.google.com/github/romybeaute/hypnomed/blob/main/all_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Load participant list

In [23]:
import numpy as np
import requests
import os

!pip install nilearn
import nibabel as nib
import numpy as np
import os, glob
import pandas as pd
from scipy.io import savemat
from scipy.io import loadmat
from nilearn import surface


# Function to download and read text file containing subject list
def download_and_load_text(url):
    response = requests.get(url)
    response.raise_for_status()
    data = response.text.splitlines()
    return data


# URL of the subject list text file
excluded_subject = ['sub-32']
subject_list_url = 'https://raw.githubusercontent.com/romybeaute/hypnomed/main/diffusion_embedding/scripts_reordered/subject_list.txt'
subject_list = [sub for sub in download_and_load_text(subject_list_url)  if sub not in excluded_subject]


print(f"n={len(subject_list)} subjects")

n=39 subjects


Load individual embeddings

In [39]:

def download_file(url, filename):
    """
    Helper function to download a file from a URL.
    """
    response = requests.get(url)
    response.raise_for_status()

    # Write the content to a file
    with open(filename, 'wb') as f:
        f.write(response.content)


def load_npy_files(base_url, filenames):
    """
    Downloads and loads multiple .npy files into a list of numpy arrays.
    """
    arrays = []
    for filename in filenames:
        full_path = f"{base_url}/{filename}"
        download_file(full_path, filename)  # Download file
        array = np.load(filename)  # Load the .npy file
        arrays.append(array)
        os.remove(filename)  # Clean up the file after loading
    return arrays



# Base URL for the directory containing .npy files
base_url = 'https://raw.githubusercontent.com/romybeaute/hypnomed/main/diffusion_embedding/emb_outputs/emb_output_states'



filenames = [
    filename
    for sub in subject_list if sub not in excluded_subject
    for filename in(
    f'embedding_dense_emb.{sub}.ses-001.control.npy',
    f'embedding_dense_emb.{sub}.ses-001.meditation.npy',
    f'embedding_dense_emb.{sub}.ses-001.hypnose.npy' )]

# Load all .npy files into a list of numpy arrays
all_embeddings = load_npy_files(base_url, filenames)
all_embeddings_array = np.stack(all_embeddings, axis=0)



Realignment of all embeddings : rotate and gather all subjects into a single matrix

In [72]:



def run_realign(emb, tar):
    realign = []
    for i, embed in enumerate(emb):
        u, s, v = np.linalg.svd(tar.T.dot(embed), full_matrices=False)
        xfm = v.T.dot(u.T)
        realign.append(embed.dot(xfm))
    return np.array(realign)


def load_template(template_base_url):
  #load cortex template
    cx_url = 'https://raw.githubusercontent.com/romybeaute/hypnomed/main/data/cortex.mat'
    download_file(cx_url,'cortex.mat')
    cortex = loadmat('cortex.mat')
    cortex = np.squeeze(cortex['cortex'])
    os.remove('cortex.mat')

    template = np.zeros([20484,5])
    for grad_id in range(1,6):#for each 5 gradient
        left_url = f"{template_base_url}/hcp.embed.grad_{grad_id}.L.fsa5.func.gii"
        right_url = f"{template_base_url}/hcp.embed.grad_{grad_id}.R.fsa5.func.gii"

        # Download the Gifti files
        left_filename = f"grad_{grad_id}.L.func.gii"
        right_filename = f"grad_{grad_id}.R.func.gii"
        download_file(left_url, left_filename)
        download_file(right_url, right_filename)

        # Load Gifti data
        template_L = nib.load(left_filename).darrays[0].data
        template_R = nib.load(right_filename).darrays[0].data

        # Combine left and right into the template
        template[:, grad_id - 1] = np.hstack([template_L, template_R])

        # Cleanup the downloaded Gifti files
        os.remove(left_filename)
        os.remove(right_filename)

    # Standardize the template
    template = template[cortex, :]  # Apply cortex mask
    template = (template - np.mean(template, axis=0)) / np.std(template, axis=0)

    return template

template_base_url = 'https://raw.githubusercontent.com/romybeaute/hypnomed/main/data/template'
template = load_template(template_base_url)
template.shape #shape : (n_voxels in template, n gradients)

(18715, 5)

In [74]:
all_embeddings_array = np.stack(all_embeddings, axis=0)
print(all_embeddings_array.shape)

realigned = run_realign(all_embeddings_array, template)

all_embs = savemat('all_embedding.mat', mdict={'emb': realigned, 'subs': np.array(subject_list, dtype='str')})
print('Group matrix succeded and saved in {}'.format('all_embedding.mat'))


(117, 18715, 5)


In [80]:

f = loadmat('all_embedding.mat')
print('Embeddings shape:', f['emb'].shape)
print(f'n subjects =', len(f['subs']))


Embeddings shape: (117, 18715, 5)
n subjects = 39
